In [ ]:
!pip install -U pyspark

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import desc
from pyspark.sql.functions import *
from pyspark.sql.functions import max as sparkMax
from IPython.display import display
import pyspark.sql.functions as F

#importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix

import warnings; warnings.filterwarnings(action='ignore')

In [ ]:
SparkSession \
  .builder \
  .master("local[*]")\
  .appName("Pyspark") \
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "16g") \
  .config("spark.driver.memory", "16g")\
  .config("spark.sql.shuffle.partitions" , "800") \
  .config("spark.memory.offHeap.enabled",'true')\
  .config("spark.memory.offHeap.size","16g")\
  .getOrCreate()

In [ ]:
#Creating a Spark session
mySpark = SparkSession.builder.getOrCreate()
spark = SparkSession(mySpark)

In [ ]:
#loading the dataset
data = spark.read.csv("marketing/data/bank_full_raw.csv", sep = ';', inferSchema="true", header="true")

In [ ]:
#Display of column name and data type
data.printSchema()

# 1. Exploratory Data Analysis

In [ ]:
data.count()

In [ ]:
data.show(10)

In [ ]:
#Removal of rows & columns with NaN/Null values and duplicates
data.dropna()
data.dropDuplicates()
data.count()

In [ ]:
#check the null column count using sql
from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(a) | col(a).isNull(), a)).alias(a) for a in data.columns]).show()

In [ ]:
df = data
df = df.replace('yes','1')
df = df.replace('no','0')
df = df.withColumn("default",df.default.cast('integer'))
df = df.withColumn("loan",df.loan.cast('integer'))
df = df.withColumn("housing",df.housing.cast('integer'))
df = df.withColumn("y",df.y.cast('integer'))

In [ ]:
#Display of column name and data type
df.printSchema()

In [ ]:
data.groupBy("month").count().show() 

In [ ]:
data.groupBy("job").count().show() 

In [ ]:
data.groupBy("marital").count().show()

In [ ]:
data.groupBy("education").count().show() 

In [ ]:
data.groupBy("housing").count().show()  

In [ ]:
data.groupBy("loan").count().show()

In [ ]:
data.groupBy("contact").count().show()

In [ ]:
data.groupBy("campaign").count().show()

In [ ]:
data.groupBy("poutcome").count().show()  

In [ ]:
data.groupBy("y").count().show() 

In [ ]:
data.filter((F.col('poutcome')=='unknown'))\
    .filter(
        (F.col('y') == 'yes')
    )\
    .show()

In [ ]:
data.filter((F.col('poutcome')=='success'))\
    .filter(
        (F.col('y') == 'yes')
    )\
    .show()

In [ ]:
data.filter((F.col('poutcome')=='failure'))\
    .filter(
        (F.col('y') == 'yes')
    )\
    .show()

In [ ]:
sns.set(style="ticks")

sns.pairplot(data.toPandas())
plt.show()

In [ ]:
df.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="job", y = "y", data = df.toPandas())
sns.despine(left = True, bottom = True)
ax.set(xlabel='job', ylabel='y')

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="marital", y = "y", data = df.toPandas())
sns.despine(left = True, bottom = True)
ax.set(xlabel='marital', ylabel='y')

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="education", y = "y", data = df.toPandas())
sns.despine(left = True, bottom = True)
ax.set(xlabel='education', ylabel='y')

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="housing", y = "y", data = df.toPandas())
sns.despine(left = True, bottom = True)
ax.set(xlabel='housing', ylabel='y')

# 2. Data Preperation

In [ ]:
cols = df.columns

#### Encode catagorical features and transform/merge multiple columns into a vector column

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'y', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age', 'default', 'balance', 'housing', 'loan', 'day', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="Subscribed")
stages += [assembler]

#### Notice that the dataframe now has 2 new columns in the beginning - "label" and "Subscribed"

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'Subscribed'] + cols
df = df.select(selectedCols)
df.printSchema()

In [ ]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

#### 80/20 train/test data split 

In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed = 2022)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'Subscribed', labelCol = 'label', maxIter=15)
lrModel = lr.fit(train)

In [ ]:
beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta-Coefficients')
plt.show()

In [ ]:
trainingSummary = lrModel.summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

In [ ]:
pr = trainingSummary.pr.toPandas()
plt.plot(pr['recall'],pr['precision'])
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()

In [ ]:
predictions = lrModel.transform(test)
predictions.filter((F.col('prediction') == 1.0))\
    .show()

In [ ]:
predictions.filter((F.col('prediction') == 1.0))\
    .count()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

## Decision Tree Classifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'Subscribed', labelCol = 'label', maxDepth= 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.filter((F.col('prediction') == 1.0))\
    .show()

In [ ]:
predictions.filter((F.col('prediction') == 1.0))\
    .count()

In [ ]:
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

## Random Forest Classifier

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'Subscribed', labelCol = 'label')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.filter((F.col('prediction') == 1.0))\
    .show()

In [ ]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions)))

## Gradient-Boosted Tree Classifier

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol = 'Subscribed', labelCol = 'label', maxIter=10)
gbtModel = gbt.fit(train)
predictions = gbtModel.transform(test)
predictions.filter((F.col('prediction') == 1.0))\
    .show()

In [ ]:
predictions.filter((F.col('prediction') == 1.0))\
    .count()

In [ ]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions)))

## Modifying this model with the ParamGridBuilder as well as the CrossValidator because the Logistic Regression produced the best results

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder().addGrid(lr.maxIter, [500]) \
                                .addGrid(lr.regParam, [0]) \
                                .addGrid(lr.elasticNetParam, [1]) \
                                .build())
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

In [ ]:
predictions.show()

In [ ]:
predictions.count()